This is **not** a generic evaluation notebook.

- LLaMA model only
- Price Tagging Game only

You need to follow this notebook and create new one for your experiments.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, LlamaTokenizer
import random
from tqdm import tqdm
import re, torch
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from models.llama.modelings_alignable_llama import *
from counterfactual_datasets.price_tagging_game import *
import pickle

#### Loading Alpaca-test

In [ ]:
alignment_config = {
    'layer': 15,
    "token_range" : [81, 82]
}
model = AlignableLlamaForCausalLM.from_pretrained(
    "../../alpaca_test/",
    alignment_config=alignment_config,
    torch_dtype=torch.bfloat16,
)
_ = model.to("cuda")

#### Loading Alpaca-7B

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="../../alpaca_7b/",
    cache_dir=CACHE_DIR
)

In [ ]:
alignment_config = {
    'layer': 15,
    "token_range" : [81, 82]
}
model = AlignableLlamaForCausalLM.from_pretrained(
    "../../alpaca_7b/",
    alignment_config=alignment_config,
    torch_dtype=torch.bfloat16
)
_ = model.to("cuda")

In [ ]:
alpaca_prompt_template = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Input:
%s

### Response:
"""

alpaca_prompt_template_no_inputs = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
%s

### Response:
"""

In [ ]:
alpaca_instruction = """Please say yes only if Sam is heavier than John, otherwise no."""
prompt = alpaca_prompt_template % (alpaca_instruction, "Sam weigh 112 lbs, John weigh 163 lbs")

# alpaca_instruction = """Does Donald Trump use computer?"""
# prompt = alpaca_prompt_template_no_inputs % (alpaca_instruction)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.to("cuda")
model.eval()
outputs = model(
    input_ids,
    attention_mask=attention_mask
)
pred_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
generated_tokens = tokenizer.decode(pred_labels[0])

afc_1 = tokenizer.convert_tokens_to_ids("Yes")
afc_2 = tokenizer.convert_tokens_to_ids("No")
afc_1_prob = outputs.logits[:, -1][0][afc_1]
afc_2_prob = outputs.logits[:, -1][0][afc_2]
if afc_1_prob > afc_2_prob:
    afc = "Yes"
else:
    afc = "No"
print(f"afc label = {afc} ({afc_1_prob}/{afc_2_prob}) ; pred label = {generated_tokens}")
    

#### Factual: Pricing Tag Game

In [ ]:
raw_prealign = factual_sampler(
    tokenizer,
    5000,
    game="pricing_tag"
)
prealign_dataset = Dataset.from_dict(
    {
        "input_ids": raw_prealign[0], 
        "labels": raw_prealign[1],
    }
).with_format("torch")
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [ ]:
total_count = 0
correct_count = 0
model.eval()
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        outputs = model(
            input_ids=inputs['input_ids'],
            labels=inputs['labels'],
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

#### (Experimental) Factual: Pricing Tag Game Counterfactual

In [ ]:
checkpoint_state_dict = torch.load("../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.42.intl.15.intr.81.82/pytorch-rotate-best.bin")
baselining = True
if baselining:
    print("Baselining with a random rotation + learned boundary")
    n = model.model.rotate_layer.parametrizations.weight.original.shape[0]
    rand_weight = torch.empty(n,n).to("cuda").to(torch.bfloat16)
    torch.nn.init.orthogonal_(rand_weight)
    model.model.rotate_layer.parametrizations.weight.original.data = rand_weight.data
else:
    model.model.rotate_layer.load_state_dict(
        checkpoint_state_dict['rotate_layer']
    )
model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=2.51,
    bound_width=2.00,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=8,
)

In [ ]:
total_count = 0
correct_count = 0
with torch.no_grad():
    for step, inputs in enumerate(tqdm(test_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        source_hidden_states = model(
            input_ids=inputs['source_input_ids'],
            output_rotated_hidden_states_only=True
        ).rotated_hidden_states
        outputs = model(
            input_ids=inputs['input_ids'],
            source_hidden_states=source_hidden_states,
            intervention_ids=inputs['intervention_ids'],
            labels=inputs['labels']
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()

current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

#### Goal: we need to find two good bracket settings to study 0-shot transfer!
2.51 to 5.51 seems to be working with 0.95

5.49 to 8.49 seems to be working with 0.94

In [ ]:
raw_prealign = factual_sampler(
    tokenizer,
    500,
    game="pricing_tag",
    amount=None,
    lower_bound=2.51,
    bound_width=4.00,
)
prealign_dataset = Dataset.from_dict(
    {
        "input_ids": raw_prealign[0], 
        "labels": raw_prealign[1],
    }
).with_format("torch")
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [ ]:
total_count = 0
correct_count = 0
model.eval()
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        outputs = model(
            input_ids=inputs['input_ids'],
            labels=inputs['labels'],
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

#### Tokenization

In [ ]:
prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Please say yes only if it costs between 1.23 and 4.56 dollars, otherwise no.

### Input:
7.89 dollars

### Response:
"""

In [ ]:
tokenizer.tokenize(prompt)[68:]

#### Get two evaluation set each with 1K examples
one for models getting correct
one for models getting wrong

In [ ]:
incorrect_triples = set([])
correct_triples = set([])

In [ ]:
model.eval()
with torch.no_grad():
    pbar = tqdm(range(500))
    for i in pbar:
        input_ids, output_ids, triple = pricing_tag_game_example_sampler_with_info(
            tokenizer, None, None, None
        )
        input_ids = torch.tensor(input_ids).unsqueeze(dim=0).to(model.device)
        output_ids = torch.tensor(output_ids).unsqueeze(dim=0).to(model.device)
        # aligning forward!
        outputs = model(
            input_ids=input_ids,
            labels=output_ids,
        )
        actual_test_labels = output_ids[0, -1]
        pred_test_labels = torch.argmax(outputs.logits[0, -1], dim=-1)
        if actual_test_labels == pred_test_labels:
            correct_triples.add(triple)
        else:
            incorrect_triples.add(triple)
        pbar.set_description("#correct %s ; #incorrect %s" % (
            len(correct_triples), len(incorrect_triples)))

In [ ]:
incorrect_triples_sampled = random.sample(list(incorrect_triples), k=1000)
correct_triples_sampled = random.sample(list(correct_triples), k=1000)

#### Re-eval with correct and incorrect triples as well as learned DAS

In [ ]:
# Open the pickled file for binary reading
with open('../logs/consistency_triples.pickle', 'rb') as file:
    # Load the pickled object from the file
    loaded_object = pickle.load(file)
incorrect_triples = loaded_object["incorrect_triples"]
correct_triples = loaded_object["correct_triples"]

incorrect_triples_by_regions = {
    1 : set([]),
    2 : set([]),
    3 : set([]),
}
correct_triples_by_regions = {
    1 : set([]),
    2 : set([]),
    3 : set([]),
}

for t in incorrect_triples:
    if t[-1] < t[0]:
        incorrect_triples_by_regions[1].add(t)
    elif t[-1] > t[1]:
        incorrect_triples_by_regions[3].add(t)
    else:
        incorrect_triples_by_regions[2].add(t)

for t in correct_triples:
    if t[-1] < t[0]:
        correct_triples_by_regions[1].add(t)
    elif t[-1] > t[1]:
        correct_triples_by_regions[3].add(t)
    else:
        correct_triples_by_regions[2].add(t)

In [ ]:
correct_results = {}
incorrect_results = {}

In [ ]:
raw_data = bound_alignment_sampler_with_triples(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler_with_triples,
        upper_bound_alignment_example_sampler_with_triples
    ],
    correct_triples_by_regions
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
# for seed in [42]:
#     for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
#         for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
#             end_idx = token_idx + 1
#             print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
#             checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
#             model.model.rotate_layer.load_state_dict(
#                 checkpoint_state_dict['rotate_layer']
#             )
#             model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
#             model.alignment_config = {
#                 'layer': layer_idx,
#                 "token_range" : [token_idx, end_idx]
#             }
#             model.model.alignment_config = {
#                 'layer': layer_idx,
#                 "token_range" : [token_idx, end_idx]
#             }
            
#             total_count = 0
#             correct_count = 0
#             with torch.no_grad():
#                 for step, inputs in enumerate(tqdm(test_dataloader)):
#                     for k, v in inputs.items():
#                         if v is not None and isinstance(v, torch.Tensor):
#                             inputs[k] = v.to(model.device)

#                     # aligning forward!
#                     source_hidden_states = model(
#                         input_ids=inputs['source_input_ids'],
#                         output_rotated_hidden_states_only=True
#                     ).rotated_hidden_states
#                     outputs = model(
#                         input_ids=inputs['input_ids'],
#                         source_hidden_states=source_hidden_states,
#                         intervention_ids=inputs['intervention_ids'],
#                         labels=inputs['labels']
#                     )

#                     actual_test_labels = inputs['labels'][:, -1]
#                     pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
#                     correct_labels = (actual_test_labels==pred_test_labels)

#                     total_count += len(correct_labels)
#                     correct_count += correct_labels.sum().tolist()

#             current_acc = round(correct_count/total_count, 2)
#             correct_results[(layer_idx, token_idx)] = current_acc
#             print(f"IIA accuracy: {current_acc}")

raw_data = bound_alignment_sampler_with_triples(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler_with_triples,
        upper_bound_alignment_example_sampler_with_triples
    ],
    incorrect_triples_by_regions
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [42]:
    for token_idx in [77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            incorrect_results[(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")



In [ ]:
with open('../logs/eval_results_consistency_seed_42.pkl', 'wb') as file:
    pickle.dump(
        {"correct_results" : correct_results, 
         "incorrect_results" : incorrect_results}
        , file)

#### Finding Irrelevant Context Model Can Still Perform

"Pricing tag game!" (84%)

"LLaMA is not conscious." (84%)

"Fruitarian Frogs May Be Doing Flowers a Favor" (85%; citation: NYT America 04/28 First Story)

In [ ]:
def pricing_tag_game_example_sampler_with_context(
    tokenizer,
    context,
    amount,
    lower_bound,
    bound_width,
):
    lower_bound_sample, upper_bound_sample, amount_sample = pricing_tag_game_config_sampler(
        amount,
        lower_bound,
        bound_width
    )
    lower_bound_str = "%.2f" % lower_bound_sample
    upper_bound_str = "%.2f" % upper_bound_sample
    if amount_sample >= float(lower_bound_str) and amount_sample <= float(upper_bound_str):
        label = tokenizer.convert_tokens_to_ids("Yes")
    else:
        label = tokenizer.convert_tokens_to_ids("No")

    amount_str = "%.2f dollars" % amount_sample
    instruction = f"{context} Please say yes only if it costs between {lower_bound_str} and {upper_bound_str} dollars, otherwise no."
    alpaca_prompt = alpaca_prompt_template % (instruction, amount_str)

    input_ids = tokenizer(alpaca_prompt, return_tensors="pt").input_ids[0]
    output_ids = (torch.ones(input_ids.shape[0])*-100).long().tolist()
    output_ids[-1] = label
    input_ids = input_ids.tolist()
    
    return input_ids, output_ids

def factual_sampler_with_context(
    tokenizer,
    context,
    max_n_training_examples,
    game="pricing_tag",
    amount=None,
    lower_bound=None,
    bound_width=None,
):
    
    all_input_ids = []
    all_output_ids = [] # this one does not have input ids, etc..
    for _ in range(max_n_training_examples):
        if "pricing_tag" in game:
            input_ids, output_ids = pricing_tag_game_example_sampler_with_context(
                tokenizer,
                context,
                amount,
                lower_bound,
                bound_width
            )
        elif game == "continent_retrieval":
            pass
        all_input_ids += [input_ids]
        all_output_ids += [output_ids]
        
    return all_input_ids, all_output_ids




In [ ]:
raw_prealign = factual_sampler_with_context(
    tokenizer,
    "Fruitarian Frogs May Be Doing Flowers a Favor",
    500,
    game="pricing_tag"
)
prealign_dataset = Dataset.from_dict(
    {
        "input_ids": raw_prealign[0], 
        "labels": raw_prealign[1],
    }
).with_format("torch")
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

total_count = 0
correct_count = 0
model.eval()
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        outputs = model(
            input_ids=inputs['input_ids'],
            labels=inputs['labels'],
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

In [ ]:
def bound_alignment_sampler_with_context(
    tokenizer,
    base_context,
    source_context,
    max_n_training_examples,
    bound_functors,
    amount=None,
    lower_bound=None,
    bound_width=None,
):
    all_base_input_ids = []
    all_source_input_ids = []
    all_ctf_output_ids = [] # this one does not have input ids, etc..
    all_intervention_ids = []
    
    for _ in range(max_n_training_examples):
        bound_functor = random.choice(bound_functors)
        base_lower_bound_sample, base_upper_bound_sample, \
            source_lower_bound_sample, source_upper_bound_sample, \
            base_amount_sample, source_amount_sample, \
            ctf_label, ctf_label_str = bound_functor(
                tokenizer,
                amount,
                lower_bound,
                bound_width,
            )

        base_amount_str = "%.2f dollars" % base_amount_sample
        source_amount_str = "%.2f dollars" % source_amount_sample
        base_lower_bound_str = "%.2f" % base_lower_bound_sample
        base_upper_bound_str = "%.2f" % base_upper_bound_sample
        source_lower_bound_str = "%.2f" % source_lower_bound_sample
        source_upper_bound_str = "%.2f" % source_upper_bound_sample
        
        # print(f"base: [{base_lower_bound_str}, {base_upper_bound_str}], {base_amount_str}")
        # print(f"source: [{source_lower_bound_str}, {source_upper_bound_str}], {source_amount_str}")
        # print(f"ctf label: {ctf_label_str}")
        
        base_instruction = f"{base_context} Please say yes only if it costs between {base_lower_bound_str} and {base_upper_bound_str} dollars, otherwise no."
        source_instruction = f"{source_context} Please say yes only if it costs between {source_lower_bound_str} and {source_upper_bound_str} dollars, otherwise no."
        
        base_alpaca_prompt = alpaca_prompt_template % (base_instruction, base_amount_str)
        source_alpaca_prompt = alpaca_prompt_template % (source_instruction, source_amount_str)
        
        base_input_ids = tokenizer(base_alpaca_prompt, return_tensors="pt").input_ids[0]
        source_input_ids = tokenizer(source_alpaca_prompt, return_tensors="pt").input_ids[0]
        base_input_ids = base_input_ids.tolist()
        source_input_ids = source_input_ids.tolist()
        ctf_output_ids = (torch.ones(len(base_input_ids))*-100).long().tolist()
        ctf_output_ids[-1] = ctf_label
        intervention_id = 0 if bound_functor == bound_functors[0] else 1
        
        all_base_input_ids += [base_input_ids]
        all_source_input_ids += [source_input_ids]
        
        all_ctf_output_ids += [ctf_output_ids]
        all_intervention_ids += [intervention_id]
        
    return all_base_input_ids, all_source_input_ids, all_ctf_output_ids, all_intervention_ids

In [ ]:
irrelevant_context_results = {
    "Pricing tag game!" : dict(),
    "Fruitarian Frogs May Be Doing Flowers a Favor" : dict(),
}

In [ ]:
context = "Pricing tag game!" # +6

# context = "Fruitarian Frogs May Be Doing Flowers a Favor" # +15

raw_data = bound_alignment_sampler_with_context(
    tokenizer,
    context,
    context,
    1000,
    [
        lower_bound_alignment_example_sampler,
        upper_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
if context == "Pricing tag game!":
    offset = 6
elif context == "Fruitarian Frogs May Be Doing Flowers a Favor":
    offset = 15
print(f"Adding prefix: {context}")
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            token_idx_re = token_idx + offset
            end_idx_re = token_idx_re + 1
            end_idx = token_idx + 1
            print(f"eval with reindexed token_idx = {token_idx_re} ({token_idx}) and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx_re, end_idx_re]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx_re, end_idx_re]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            irrelevant_context_results[context][(layer_idx, token_idx_re)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
# context = "Pricing tag game!" # +6

context = "Fruitarian Frogs May Be Doing Flowers a Favor" # +15

raw_data = bound_alignment_sampler_with_context(
    tokenizer,
    context,
    context,
    1000,
    [
        lower_bound_alignment_example_sampler,
        upper_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
if context == "Pricing tag game!":
    offset = 6
elif context == "Fruitarian Frogs May Be Doing Flowers a Favor":
    offset = 15
print(f"Adding prefix: {context}")
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            token_idx_re = token_idx + offset
            end_idx_re = token_idx_re + 1
            end_idx = token_idx + 1
            print(f"eval with reindexed token_idx = {token_idx_re} ({token_idx}) and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx_re, end_idx_re]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx_re, end_idx_re]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            irrelevant_context_results[context][(layer_idx, token_idx_re)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
with open('../logs/eval_results_irrelevant_context_seed_77.pkl', 'wb') as file:
    pickle.dump(
        irrelevant_context_results, file)

#### Eval test the main experiments

In [ ]:
main_eval_results = {
    "lower_bound_alignment" : dict(),
    "both_bound_alignment" : dict(),
    "midpoint_alignment" : dict(),
    "bracket_alignment" : dict(),
}
control = False
if control:
    original_weight = torch.empty(4096, 4096).to(torch.bfloat16).to(model.device)
    orth_weights = torch.nn.init.orthogonal_(weight)

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lb.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            if control:
                model.model.rotate_layer.weight.data = orth_weights
                model.model.rotate_layer.parametrizations.weight.original.data = original_weight
            else:
                model.model.rotate_layer.load_state_dict(
                    checkpoint_state_dict['rotate_layer']
                )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            main_eval_results['lower_bound_alignment'][(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler,
        upper_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-last.bin")
            if control:
                model.model.rotate_layer.weight.data = orth_weights
                model.model.rotate_layer.parametrizations.weight.original.data = original_weight
            else:
                model.model.rotate_layer.load_state_dict(
                    checkpoint_state_dict['rotate_layer']
                )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            main_eval_results['both_bound_alignment'][(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
raw_data = midpoint_alignment_sampler(
    tokenizer,
    1000,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_mid_diff.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            if control:
                model.model.rotate_layer.weight.data = orth_weights
                model.model.rotate_layer.parametrizations.weight.original.data = original_weight
            else:
                model.model.rotate_layer.load_state_dict(
                    checkpoint_state_dict['rotate_layer']
                )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            main_eval_results['midpoint_alignment'][(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
raw_data = bracket_alignment_sampler(
    tokenizer,
    1000,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_bracket.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            if control:
                model.model.rotate_layer.weight.data = orth_weights
                model.model.rotate_layer.parametrizations.weight.original.data = original_weight
            else:
                model.model.rotate_layer.load_state_dict(
                    checkpoint_state_dict['rotate_layer']
                )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            main_eval_results['bracket_alignment'][(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
with open('../logs/eval_main_results_seed_77.pkl', 'wb') as file:
    pickle.dump(
        main_eval_results, file)

#### Zero-shot Transfer between two fixed bounds DAS learning

2.51 to 5.51 seems to be working with 0.95

5.49 to 8.49 seems to be working with 0.94

In [ ]:
zero_shot_results = {
    "original" : dict(),
    "transfer" : dict(),
}

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=2.51,
    bound_width=3.00,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_fixed.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            zero_shot_results['transfer'][(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=5.49,
    bound_width=3.00,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_fixed.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            zero_shot_results['original'][(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
with open('../logs/eval_zero_shot_results_seed_77.pkl', 'wb') as file:
    pickle.dump(
        zero_shot_results, file)

#### Zero-shot sibiling instruction transfer

In [ ]:
def pricing_tag_game_example_sampler_with_different_return(
    tokenizer,
    new_labels,
    amount,
    lower_bound,
    bound_width,
):
    lower_bound_sample, upper_bound_sample, amount_sample = pricing_tag_game_config_sampler(
        amount,
        lower_bound,
        bound_width
    )
    lower_bound_str = "%.2f" % lower_bound_sample
    upper_bound_str = "%.2f" % upper_bound_sample
    if amount_sample >= float(lower_bound_str) and amount_sample <= float(upper_bound_str):
        label = tokenizer.convert_tokens_to_ids(new_labels[0])
    else:
        label = tokenizer.convert_tokens_to_ids(new_labels[1])

    amount_str = "%.2f dollars" % amount_sample
    instruction = f"Please say {new_labels[0]} only if it costs between {lower_bound_str} and {upper_bound_str} dollars, otherwise {new_labels[1]}."
    alpaca_prompt = alpaca_prompt_template % (instruction, amount_str)

    input_ids = tokenizer(alpaca_prompt, return_tensors="pt").input_ids[0]
    output_ids = (torch.ones(input_ids.shape[0])*-100).long().tolist()
    output_ids[-1] = label
    input_ids = input_ids.tolist()
    
    return input_ids, output_ids

def factual_sampler_with_different_return(
    tokenizer,
    new_labels,
    max_n_training_examples,
    game="pricing_tag",
    amount=None,
    lower_bound=None,
    bound_width=None,
):
    
    all_input_ids = []
    all_output_ids = [] # this one does not have input ids, etc..
    for _ in range(max_n_training_examples):
        if "pricing_tag" in game:
            input_ids, output_ids = pricing_tag_game_example_sampler_with_different_return(
                tokenizer,
                new_labels,
                amount,
                lower_bound,
                bound_width
            )
        elif game == "continent_retrieval":
            pass
        all_input_ids += [input_ids]
        all_output_ids += [output_ids]
        
    return all_input_ids, all_output_ids

def bound_alignment_sampler_with_different_return(
    tokenizer,
    new_labels,
    max_n_training_examples,
    bound_functors,
    amount=None,
    lower_bound=None,
    bound_width=None,
):
    all_base_input_ids = []
    all_source_input_ids = []
    all_ctf_output_ids = [] # this one does not have input ids, etc..
    all_intervention_ids = []
    
    for _ in range(max_n_training_examples):
        bound_functor = random.choice(bound_functors)
        base_lower_bound_sample, base_upper_bound_sample, \
            source_lower_bound_sample, source_upper_bound_sample, \
            base_amount_sample, source_amount_sample, \
            ctf_label, ctf_label_str = bound_functor(
                tokenizer,
                amount,
                lower_bound,
                bound_width,
            )
        # overwrite a little
        if ctf_label_str == "Yes":
            ctf_label_str = new_labels[0]
            ctf_label = tokenizer.convert_tokens_to_ids(new_labels[0])
        elif ctf_label_str == "No":
            ctf_label_str = new_labels[1]
            ctf_label = tokenizer.convert_tokens_to_ids(new_labels[1])

        base_amount_str = "%.2f dollars" % base_amount_sample
        source_amount_str = "%.2f dollars" % source_amount_sample
        base_lower_bound_str = "%.2f" % base_lower_bound_sample
        base_upper_bound_str = "%.2f" % base_upper_bound_sample
        source_lower_bound_str = "%.2f" % source_lower_bound_sample
        source_upper_bound_str = "%.2f" % source_upper_bound_sample
        
        # print(f"base: [{base_lower_bound_str}, {base_upper_bound_str}], {base_amount_str}")
        # print(f"source: [{source_lower_bound_str}, {source_upper_bound_str}], {source_amount_str}")
        # print(f"ctf label: {ctf_label_str}")
        
        base_instruction = f"Please say {new_labels[0]} only if it costs between {base_lower_bound_str} and {base_upper_bound_str} dollars, otherwise {new_labels[1]}."
        source_instruction = f"Please say {new_labels[0]} only if it costs between {source_lower_bound_str} and {source_upper_bound_str} dollars, otherwise {new_labels[1]}."
        
        base_alpaca_prompt = alpaca_prompt_template % (base_instruction, base_amount_str)
        source_alpaca_prompt = alpaca_prompt_template % (source_instruction, source_amount_str)
        
        base_input_ids = tokenizer(base_alpaca_prompt, return_tensors="pt").input_ids[0]
        source_input_ids = tokenizer(source_alpaca_prompt, return_tensors="pt").input_ids[0]
        base_input_ids = base_input_ids.tolist()
        source_input_ids = source_input_ids.tolist()
        ctf_output_ids = (torch.ones(len(base_input_ids))*-100).long().tolist()
        ctf_output_ids[-1] = ctf_label
        intervention_id = 0 if bound_functor == bound_functors[0] else 1
        
        all_base_input_ids += [base_input_ids]
        all_source_input_ids += [source_input_ids]
        
        all_ctf_output_ids += [ctf_output_ids]
        all_intervention_ids += [intervention_id]
        
    return all_base_input_ids, all_source_input_ids, all_ctf_output_ids, all_intervention_ids

In [ ]:
raw_prealign = factual_sampler_with_different_return(
    tokenizer,
    ['True', 'False'],
    500,
    game="pricing_tag"
)
prealign_dataset = Dataset.from_dict(
    {
        "input_ids": raw_prealign[0], 
        "labels": raw_prealign[1],
    }
).with_format("torch")
prealign_dataloader = DataLoader(
    prealign_dataset, batch_size=8
)

In [ ]:
total_count = 0
correct_count = 0
model.eval()
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        # aligning forward!
        outputs = model(
            input_ids=inputs['input_ids'],
            labels=inputs['labels'],
        )

        actual_test_labels = inputs['labels'][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = (actual_test_labels==pred_test_labels)

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count/total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

In [ ]:
different_return_results = {}

In [ ]:
raw_data = bound_alignment_sampler_with_different_return(
    tokenizer,
    ["True", "False"],
    1000,
    [
        lower_bound_alignment_example_sampler,
        upper_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=128,
)

# grid search
# [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [77]:
    for token_idx in [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]:
        for layer_idx in [0, 5, 10, 15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-best.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            different_return_results[(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
with open('../logs/eval_different_return_results_seed_77.pkl', 'wb') as file:
    pickle.dump(
        different_return_results, file)

#### Zero-shot between type DAS

In [ ]:
def lower_bound_alignment_example_reversed_sampler(
    tokenizer,
    amount=None,
    lower_bound=None,
    bound_width=None
):
    base_lower_bound_sample, base_upper_bound_sample, _ = \
        pricing_tag_game_config_sampler(
            amount,
            lower_bound,
            bound_width
        )
    source_lower_bound_sample, source_upper_bound_sample, _ = \
        pricing_tag_game_config_sampler(
            amount,
            lower_bound,
            bound_width
        )
    
    ctf_label_str = random.choice(["Yes", "No"])
    if ctf_label_str == "Yes":
        ctf_label = tokenizer.convert_tokens_to_ids("Yes")
        base_source_regions = [
            [1,1],
            [1,2],
            [2,2],
        ]
    elif ctf_label_str == "No":
        ctf_label = tokenizer.convert_tokens_to_ids("No")
        base_source_regions = [
            [1,3],
            [2,1],
            [2,3],
            [3,1],
            [3,2],
            [3,3]
        ]
    base_source_region = random.choice(base_source_regions)
    base_region = base_source_region[0]
    source_region = base_source_region[1]

    base_amount_sample = sample_with_region(
        base_region, base_lower_bound_sample, base_upper_bound_sample)
    source_amount_sample = sample_with_region(
        source_region, source_lower_bound_sample, source_upper_bound_sample)
        
    return base_lower_bound_sample, base_upper_bound_sample, \
        source_lower_bound_sample, source_upper_bound_sample, \
        base_amount_sample, source_amount_sample, ctf_label, ctf_label_str
    
def upper_bound_alignment_example_reversed_sampler(
    tokenizer,
    amount=None,
    lower_bound=None,
    bound_width=None
):
    base_lower_bound_sample, base_upper_bound_sample, base_amount_sample = \
        pricing_tag_game_config_sampler(
            amount,
            lower_bound,
            bound_width
        )
    source_lower_bound_sample, source_upper_bound_sample, source_amount_sample = \
        pricing_tag_game_config_sampler(
            amount,
            lower_bound,
            bound_width
        )
    
    ctf_label_str = random.choice(["Yes", "No"])
    if ctf_label_str == "Yes":
        ctf_label = tokenizer.convert_tokens_to_ids("Yes")
        base_source_regions = [
            [3,3],
            [3,2],
            [2,2],
        ]
    elif ctf_label_str == "No":
        ctf_label = tokenizer.convert_tokens_to_ids("No")
        base_source_regions = [
            [1,1],
            [1,2],
            [1,3],
            [2,1],
            [2,3],
            [3,1]
        ]
    base_source_region = random.choice(base_source_regions)
    base_region = base_source_region[0]
    source_region = base_source_region[1]
    
    base_amount_sample = sample_with_region(
        base_region, base_lower_bound_sample, base_upper_bound_sample)
    source_amount_sample = sample_with_region(
        source_region, source_lower_bound_sample, source_upper_bound_sample)
    
    return base_lower_bound_sample, base_upper_bound_sample, \
        source_lower_bound_sample, source_upper_bound_sample, \
        base_amount_sample, source_amount_sample, ctf_label, ctf_label_str

In [ ]:
type_transfer_results = {}

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    1000,
    [
        lower_bound_alignment_example_reversed_sampler,
        upper_bound_alignment_example_reversed_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0], 
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset, batch_size=8,
)

# grid search
# [69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]
# [0, 5, 10, 15, 20, 25, 30]
for seed in [42]:
    for token_idx in [81]:
        for layer_idx in [15, 20, 25, 30]:
            end_idx = token_idx + 1
            print(f"eval with token_idx = {token_idx} and layer_idx = {layer_idx}")
            checkpoint_state_dict = torch.load(f"../results_alpaca-7b/alpaca-7B.task.pricing_tag_lub.seed.{seed}.intl.{layer_idx}.intr.{token_idx}.{end_idx}/pytorch-rotate-last.bin")
            model.model.rotate_layer.load_state_dict(
                checkpoint_state_dict['rotate_layer']
            )
            model.model.intervention_boundaries.data = checkpoint_state_dict['intervention_boundaries'].data
            intervention_boundaries = model.model.intervention_boundaries.data
            intervention_boundaries = torch.clamp(intervention_boundaries, 1e-3, 1)
            start_idx_1 = 0
            end_idx_1 = int((model.model.searchable_n_embd//2) * intervention_boundaries[0])
            start_idx_2 = end_idx_1
            end_idx_2 = end_idx_1 * 2
            model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            model.model.alignment_config = {
                'layer': layer_idx,
                "token_range" : [token_idx, end_idx]
            }
            
            total_count = 0
            correct_count = 0
            with torch.no_grad():
                for step, inputs in enumerate(tqdm(test_dataloader)):
                    for k, v in inputs.items():
                        if v is not None and isinstance(v, torch.Tensor):
                            inputs[k] = v.to(model.device)

                    # aligning forward!
                    source_hidden_states = model(
                        input_ids=inputs['source_input_ids'],
                        output_rotated_hidden_states_only=True
                    ).rotated_hidden_states
                    
                    # do a hard snapped swap of aligned reprs.
                    chunk_1 = source_hidden_states[:,start_idx_1:end_idx_1].clone()
                    chunk_2 = source_hidden_states[:,start_idx_2:end_idx_2].clone()
                    source_hidden_states[:,start_idx_2:end_idx_2] = chunk_1
                    source_hidden_states[:,start_idx_1:end_idx_1] = chunk_2
                    
                    outputs = model(
                        input_ids=inputs['input_ids'],
                        source_hidden_states=source_hidden_states,
                        intervention_ids=inputs['intervention_ids'],
                        labels=inputs['labels']
                    )

                    actual_test_labels = inputs['labels'][:, -1]
                    pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)
                    correct_labels = (actual_test_labels==pred_test_labels)

                    total_count += len(correct_labels)
                    correct_count += correct_labels.sum().tolist()

            current_acc = round(correct_count/total_count, 2)
            type_transfer_results[(layer_idx, token_idx)] = current_acc
            print(f"IIA accuracy: {current_acc}")

In [ ]:
with open('../logs/eval_type_transfer_results.pkl', 'wb') as file:
    pickle.dump(
        type_transfer_results, file)

#### Dummy CLS

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    10000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
labels = [row[-1] for row in raw_data[2]]
from collections import Counter
counter = Counter(labels)
print(counter)

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    10000,
    [
        lower_bound_alignment_example_sampler,
        upper_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=None,
    bound_width=None,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
labels = [row[-1] for row in raw_data[2]]
counter = Counter(labels)
print(counter)

In [ ]:
raw_data = midpoint_alignment_sampler(
    tokenizer,
    10000,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
labels = [row[-1] for row in raw_data[2]]
counter = Counter(labels)
print(counter)

In [ ]:
raw_data = bracket_alignment_sampler(
    tokenizer,
    10000,
)
raw_test = (
    raw_data[0], 
    raw_data[1], 
    raw_data[2],
    raw_data[3]
)
labels = [row[-1] for row in raw_data[2]]
counter = Counter(labels)
print(counter)

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    10000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=2.51,
    bound_width=3.00,
)
labels = [row[-1] for row in raw_data[2]]
counter = Counter(labels)
print(counter)

In [ ]:
raw_data = bound_alignment_sampler(
    tokenizer,
    10000,
    [
        lower_bound_alignment_example_sampler,
    ],
    amount=None,
    lower_bound=5.49,
    bound_width=3.00,
)
labels = [row[-1] for row in raw_data[2]]
counter = Counter(labels)
print(counter)